<a href="https://colab.research.google.com/github/LuigiSigillo/CartoonGAN/blob/main/CartoonGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
from google.colab import drive
import os
import json
import re
import cv2
import numpy as np
from PIL import Image, ImageFilter
import sys
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow
import tensorflow as tf
!pip install tensorflow_addons
import tensorflow_addons as tfa
from tensorflow.python.client import device_lib
import keras
drive.mount('/content/drive')
#!unzip /content/drive/My\ Drive/NN/spirited_away.zip -d /content/drive/My\ Drive/NN/
#!ls /content/drive/My\ Drive/NN/
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('GPU device not found')
else:
    print('Found GPU at: {}'.format(device_name))
    print(device_lib.list_local_devices()[1])

     |████████████████████████████████| 706kB 10.8MB/s 
Mounted at /content/drive
Found GPU at: /device:GPU:0
name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11145797952
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11005459412428412042
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"



# Utils

###Save model

In [2]:
models_dir = "/content/drive/My Drive/NN/models"
checkpoint_dir = "/content/drive/My Drive/NN/models/checkpoint"

def savemodel(model,problem,checkpoint=False):
    if checkpoint:
        filename = os.path.join(checkpoint_dir, '%s.h5' %problem)
    else:
        filename = os.path.join(models_dir, '%s.h5' %problem)
    model.save(filename)
    print("\nModel saved successfully on file %s\n" %filename)

#savemodel(D,'discriminator_0')


### Load model

In [3]:
from keras.models import load_model
from keras.layers import *
models_dir = "/content/drive/My Drive/NN/models"
checkpoint_dir = "/content/drive/My Drive/NN/models/checkpoint"

def loadmodel(problem, checkpoint=False):
    if checkpoint:
        filename = os.path.join(checkpoint_dir, '%s.h5' %problem)
    else:
        filename = os.path.join(models_dir, '%s.h5' %problem)
    try:
        model = load_model(filename)
        print("\nModel loaded successfully from file %s\n" %filename)
    except OSError:    
        print("\nModel file %s not found!!!\n" %filename)
        model = None
    return model

#gen_pretrained = loadmodel(model_name)
#gen_trained = loadmodel("generator_01")
#disc_trained = loadmodel("discriminator_01")

### Load datasets

In [4]:
'''#load imgs ?
data_dir = '/content/drive/My Drive/NN/photos_rszd'
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,   validation_split=0.2, horizontal_flip=True)

train_gen = train_datagen.flow_from_directory(
        data_dir,
        subset="training",
        seed=123,
        target_size=(256, 256),
        batch_size=32
        )
validation_generator = train_datagen.flow_from_directory(
        data_dir,
        target_size=(256,256),
        batch_size=32,
        )
'''
from glob import glob
def get_dataset(dataset_name, batch_size):
    files = glob(os.path.join('/content/drive/My Drive/NN/', dataset_name, "*"))
    num_images = len(files)
    print(f"Found {num_images}  images in {dataset_name} folder.")
    ds = tf.data.Dataset.from_tensor_slices(files)
    ds = ds.shuffle(num_images)
    ds = ds.repeat()

    def fn(filename):
        x = tf.io.read_file(filename)
        x = tf.image.decode_jpeg(x, channels=3)
        img = tf.cast(x, tf.float32) / 127.5 - 1
        #print("\n tipo img = ",type(img),"\n tipo x = ",type(x), filename, type(filename))
        return img

    ds = ds.map(fn, batch_size)
    ds = ds.batch(batch_size)
    
    steps = int(np.ceil(num_images/batch_size))
    # user iter(ds) to avoid generating iterator every epoch
    return iter(ds), steps


# 1 - Image Preprocessing

### 1.1 - Resizing images


In [5]:
def resize(path):
    for item in os.listdir(path):
            im = Image.open(os.path.join(path,item))
            f, e = os.path.splitext(item)
            imResize = im.resize((256,256), Image.ANTIALIAS)
            print(f)
            imResize.save(path+"_resized/" + f + ' resized.jpg', 'JPEG', quality=90)

#!unzip /content/drive/My\ Drive/NN/photos_from_COCO.zip -d /content/drive/My\ Drive/NN/
resize('/content/drive/My Drive/NN/photos_from_COCO')

FileNotFoundError: ignored

### 1.2 Apply canny, dilate edge and gaussian

In [ ]:
def edge_smoothing(cartoon_images_filename, smoothed_images_filename):
    print("Edge-smoothing of ", cartoon_images_filename)
    origin = cv2.imread(cartoon_images_filename)
    edges = createEdgesOverlay(origin)
    result = overlayEdges(edges, origin)
    #show_images(origin, edges, result)
    result.save(smoothed_images_filename, "JPEG")

def overlayEdges(edges, origin):
    background = transformFromCV2ToPillowImageFormat(origin)
    background.paste(edges, (0, 0), edges)
    background = background.convert("RGB")
    return background

def transformFromCV2ToPillowImageFormat(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGBA)
    return Image.fromarray(img)

def createEdgesOverlay(origin):
    edges = cv2.Canny(origin, 30, 300, 3) 
    edges = cv2.dilate(edges, (3, 3))
    edges = cv2.bitwise_not(edges)
    edges = transformFromCV2ToPillowImageFormat(edges)
    makeWhiteBackgroundTransparent(edges)
    edges = edges.filter(ImageFilter.GaussianBlur) #do blurring here because doing it before making background transparent results in white halo

    return edges

def makeWhiteBackgroundTransparent(img):
    datas = img.getdata()
    newData = []
    for item in datas:
        if item[0] == 255 and item[1] == 255 and item[2] == 255:
            newData.append((255, 255, 255, 0))
        else:
            newData.append(item)
    img.putdata(newData)

def show_images(img,edges,result):
    plt.subplot(131),plt.imshow(img)
    plt.title('Original Image'), plt.xticks([]), plt.yticks([])

    plt.subplot(132),plt.imshow(edges)
    plt.title('Edge Image'), plt.xticks([]), plt.yticks([])
    
    plt.subplot(133),plt.imshow(result)
    plt.title('Result Image'), plt.xticks([]), plt.yticks([])

    plt.show()
    



path_resized = "/content/drive/My Drive/NN/spirited_away_resized/"
path_smoothed = "/content/drive/My Drive/NN/spirited_away_resized_smoothed/"


for filename in os.listdir(path_resized):
  #filename='scene43626 resized.jpg'
  f = filename.split(" ")[0] + " smoothed"
  cartoon_images_filename = path_resized + filename
  smoothed_images_filename = path_smoothed + f
  edge_smoothing(cartoon_images_filename, smoothed_images_filename)



# 2 - Initialization Phase

### Load VGG19 weights

In [6]:
from keras import applications
from keras.models import Model, Input
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D, UpSampling2D
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras import optimizers



def transferNet(input_shape):
    
    # download model
    base_model = applications.vgg19.VGG19(weights="imagenet", include_top=False, input_shape=input_shape)
    # get the output tensor from a layer of the feature extractor
    tmp_vgg_output = base_model.get_layer("block4_conv3").output
    tmp_vgg_output = Conv2D(512, (3, 3), activation='linear', padding='same',name='block4_conv4')(tmp_vgg_output)
    
    vgg = Model(inputs=base_model.input, outputs=tmp_vgg_output)
    vgg.load_weights(os.path.expanduser(os.path.join("~", ".keras", "models","vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5")), by_name=True)

    return vgg


input_shape =(256,256,3)
# load the pre-trained model
vgg = transferNet(input_shape)
vgg.summary()
!ls ~/.keras/models


80142336/80134624 [==============================] - 3s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
______________________________________________________________

#3 - GAN

### Generator

In [ ]:
from keras import applications
from keras import layers
from keras.models import Model, Input
from keras.models import Sequential
from keras.layers import *
from keras import regularizers
from keras import optimizers
import tensorflow as tf
import tensorflow_addons as tfa


def resblock(x_init):
    x = Conv2D(256, kernel_size=3, strides=1, padding='same')(x_init)
    #x = (ZeroPadding2D(padding=(1, 1)))(x)
    
    x = tfa.layers.InstanceNormalization(axis=3, center=True, scale=True)(x)
    x = tf.nn.relu(x)
    x = (Conv2D(256, kernel_size=3, strides=1, padding='same'))(x)
    #x = (ZeroPadding2D(padding=(1, 1)))(x)
    x = tfa.layers.InstanceNormalization(axis=3, center=True, scale=True)(x)    

    return x + x_init

def create_generator(input_shape):
    #model = Sequential()
    #k7n64s1
    input_t = tf.keras.layers.Input(shape=input_shape)
    
    model = (Conv2D(input_shape=input_shape, filters=64, kernel_size=7, strides=1, padding='same'))(input_t)
    #model = (ZeroPadding2D(padding=(3, 3)))(model)
    model = tfa.layers.InstanceNormalization(axis=3, center=True, scale=True)(model)
    model = tf.nn.relu(model)
    
    #### Down-Convolution
    
    #k3 n128 s2
    model = (Conv2D(filters=128, kernel_size=3, strides=2, padding='same'))(model)
    #model = (ZeroPadding2D(padding=(1, 1)))(model)
    #k3 n128 s1
    model = (Conv2D(filters=128, kernel_size=3, strides=1, padding='same'))(model)
    #model = (ZeroPadding2D(padding=(1, 1)))(model)
    
    model = tfa.layers.InstanceNormalization(axis=3, center=True, scale=True)(model)
    model = tf.nn.relu(model)


    #k3 n256 s2
    model = (Conv2D(filters=256, kernel_size=3, strides=2, padding='same'))(model)
    #model = (ZeroPadding2D(padding=(1, 1)))(model)

    #k3 n256 s1
    model = (Conv2D(filters=256, kernel_size=3, strides=1, padding='same', name="test"))(model)
    #model = (ZeroPadding2D(padding=(1, 1)))(model)
    
    model = tfa.layers.InstanceNormalization(axis=3, center=True, scale=True)(model)
    model = tf.nn.relu(model)



    #TODO
    # residual blocks
    for i in range(8):#number of res blocks
        '''model = (Conv2D(256, kernel_size=3, strides=1, padding='valid',activation="relu"))(model)
        model = (ZeroPadding2D(padding=(1, 1)))(model)
        
        model = tfa.layers.InstanceNormalization(axis=3, center=True, scale=True)(model)
        model = tf.nn.relu(model)

        model = (Conv2D(256, kernel_size=3, strides=1, padding='valid',activation="relu"))(model)
        model = (ZeroPadding2D(padding=(1, 1)))(model)
        model = tfa.layers.InstanceNormalization(axis=3, center=True, scale=True)(model)'''
        model = resblock(model)

        
        
    
    # Up-convolution
    model = (Conv2DTranspose(filters=128, kernel_size=3, strides=2, padding='same',output_padding=1))(model)
    #model = (ZeroPadding2D(padding=(1, 1)))(model)
    
    model = (Conv2D(filters=128, kernel_size=3, strides=1, padding='same'))(model)
    #model = (ZeroPadding2D(padding=(1, 1)))(model)
    
    model = tfa.layers.InstanceNormalization(axis=3, center=True, scale=True)(model)
    model = tf.nn.relu(model)
    #######################
    model = (Conv2DTranspose(filters=64, kernel_size=3, strides=2, padding='same',output_padding=1))(model)
    #model = (ZeroPadding2D(padding=(1, 1)))(model)
    
    model = (Conv2D(filters=64, kernel_size=3, strides=1, padding='same'))(model)
    #model = (ZeroPadding2D(padding=(1, 1)))(model)

    model = tfa.layers.InstanceNormalization(axis=3, center=True, scale=True)(model)
    model = tf.nn.relu(model)

    ###
    model = (Conv2D(filters=3, kernel_size=7, strides=1, padding='same'))(model)
    #model = (ZeroPadding2D(padding=(3, 3)))(model)
    model = tf.tanh(model)

    #model.compile(loss=tf.keras.losses.BinaryCrossentropy)
    model = Model(inputs=input_t, outputs=model, name="generator")

    return model

#G = create_generator(input_shape=(256,256,3))
#G.summary()


### Generator training

In [ ]:
#TODO
from glob import glob
import os
from tqdm import tqdm
import gc
import matplotlib.pyplot as plt

def pretrain_step(vgg, input_images, generator, optimizer):
    with tf.GradientTape() as tape:
        generated_images = generator(input_images)

        plt.imshow(generated_images[0])
        plt.imshow(input_images[0])
        mae = tf.keras.losses.MeanAbsoluteError()
        c_loss = mae(vgg(input_images), vgg(generated_images))

        gradients = tape.gradient(c_loss, generator.trainable_variables)
        optimizer.apply_gradients(zip(gradients, generator.trainable_variables))
        content_loss_metric = tf.keras.metrics.Mean("content_loss", dtype=tf.float32)
        content_loss_metric(c_loss)

def pretrain_generator(dataset_name,batch_size):
    summary_writer = tf.summary.create_file_writer(os.path.join('/content/drive/My Drive/NN', "pretrain"))
    print(f"Starting to pretrain generator with 10 epochs...")
    print(f"Building `{dataset_name}` dataset")
    dataset, steps_per_epoch = get_dataset(dataset_name=dataset_name, batch_size=batch_size)
    
    generator = G
    #generator(tf.keras.Input(shape=(None,256,256, 3)))
    #generator.summary()

    print("Setting up optimizer to update generator's parameters...")
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0008, beta_1=0.5)

    trained_epochs = 0
    epochs = 10

    print("Proceeding pretraining without sample images...")

    print("Starting pre-training loop, setting up summary writer to record progress on TensorBoard...")

    for epoch in range(epochs):
        epoch_idx = trained_epochs + epoch + 1

        for step in tqdm(range(1, steps_per_epoch + 1), desc=f"Pretrain Epoch {epoch + 1}/{epochs}"):
            image_batch = dataset.next()
            pretrain_step(vgg, image_batch, generator, optimizer)

            if step % 100 == 0: #100 = pretrain_reporting_steps
                global_step = (epoch_idx - 1) * steps_per_epoch + step
                with summary_writer.as_default():
                    tf.summary.scalar('content_loss', tf.keras.metrics.Mean("content_loss", dtype=tf.float32).result(), step=global_step)

                tf.keras.metrics.Mean("content_loss", dtype=tf.float32).reset_states()
        gc.collect()
    return generator
        
#gen_pretrained = pretrain_generator("photos_rszd/photos_resized", 16)


## Discriminator

In [7]:
def create_discriminator(input_shape):
    model = Sequential()
    #k7n64s1
    model.add(Conv2D(input_shape=input_shape, filters=32, kernel_size=3, strides=1, padding='same'))
    model.add(LeakyReLU())

    #k3n64s2
    model.add(Conv2D(filters=64, kernel_size=3, strides=2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    
    #k3n128s1
    model.add(Conv2D(filters=128, kernel_size=3, strides=1, padding='same'))
    model.add(tfa.layers.InstanceNormalization(axis=3, center=True, scale=True))
    model.add(LeakyReLU(alpha=0.2))
    
    #k3n128s2
    model.add(Conv2D(filters=128, kernel_size=3, strides=2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    #k3n256s1
    model.add(Conv2D(filters=256, kernel_size=3, strides=1, padding='same'))
    model.add(tfa.layers.InstanceNormalization(axis=3, center=True, scale=True))
    model.add(LeakyReLU(alpha=0.2))
    #k3n256s1
    model.add(Conv2D(filters=256, kernel_size=3, strides=1, padding='same'))
    model.add(tfa.layers.InstanceNormalization(axis=3, center=True, scale=True))
    model.add(LeakyReLU(alpha=0.2))
    #k3n1s1
    model.add(Conv2D(filters=1, kernel_size=3, strides=1, padding='same'))
   
    
    #model.compile(loss=tf.keras.losses.BinaryCrossentropy)
    return model


#D = create_discriminator(input_shape=(256,256,3))
#D.summary()  

## Adversarial model

#### Get datasets

In [8]:
from itertools import product
from imageio import imwrite,imread
import gc
from IPython.display import Image,display

def get_batches(dataset_name):
    cartoon_set,steps_per_epoch = get_dataset("datasets/"+dataset_name+"/"+dataset_name+"_resized",8)
    cartoon_smoothed_set, steps_per_epoch = get_dataset("datasets/"+dataset_name+"/"+dataset_name+"_resized_smoothed",8)
    normal_photos_set,steps_per_epoch_normal = get_dataset("datasets/normal_photos/photos_from_COCO_resized",8)

    #New dataset name = photos_from_COCO_resized
    dataset_kaggle, steps_per_epoch_coco = get_dataset(dataset_name="datasets/normal_photos/photos_from_kaggle_resized", batch_size=16)
    return cartoon_set, cartoon_smoothed_set, normal_photos_set, steps_per_epoch, dataset_kaggle


def show_and_save_images(batch_x, image_name, nrow=2, ncol=4,to_be_saved=True):
    if not isinstance(batch_x, np.ndarray):
        batch_x = batch_x.numpy()
    n, h, w, c = batch_x.shape
    out_arr = np.zeros([h * nrow, w * ncol, 3], dtype=np.uint8)
    for (i, j), k in zip(product(range(nrow), range(ncol)), range(n)):
        out_arr[(h * i):(h * (i+1)), (w * j):(w * (j+1))] = batch_x[k]
    path_name = os.path.join("/content/drive/My Drive/NN/training", DATASET_NAME,TIME+"_samples")
    check_folder(path_name)
    path_name = os.path.join(path_name,image_name)
    if to_be_saved:
        imwrite(path_name, out_arr)
    #display(Image(path_name,width=512, height=256))
    gc.collect()
    return out_arr

def check_folder(log_dir):
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    return log_dir


#show_and_save_images(tf.cast((gen_pretrained(seed, training=False) + 1) * 127.5, tf.uint8),"test.png" )

#### Loss functions

In [10]:
generator_optimizer = keras.optimizers.Adam(learning_rate=1e-5, beta_1=.5)
discriminator_optimizer = keras.optimizers.Adam(learning_rate=1e-5, beta_1=.5)

mae = tf.keras.losses.MeanAbsoluteError()
loss_bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)

@tf.function
def discriminator_loss (real, fake, real_blur):
    real_loss = loss_bce(tf.ones_like(real), real)
    fake_loss = loss_bce(tf.zeros_like(fake), fake)
    real_blur_loss = loss_bce(tf.zeros_like(real_blur), real_blur)
    #insert metrics
    d_real_loss_metric(real_loss)
    d_fake_loss_metric(fake_loss)
    d_smooth_loss_metric(real_blur_loss)
    return real_loss+fake_loss+real_blur_loss

@tf.function
def generator_loss(fake):
    return loss_bce(tf.ones_like(fake), fake)


g_total_loss_metric = tf.keras.metrics.Mean("g_total_loss", dtype=tf.float32)
g_adv_loss_metric = tf.keras.metrics.Mean("g_adversarial_loss", dtype=tf.float32)
content_loss_metric = tf.keras.metrics.Mean("content_loss", dtype=tf.float32)
style_loss_metric = tf.keras.metrics.Mean("style_loss", dtype=tf.float32)
d_total_loss_metric = tf.keras.metrics.Mean("d_total_loss", dtype=tf.float32)
d_real_loss_metric = tf.keras.metrics.Mean("d_real_loss", dtype=tf.float32)
d_fake_loss_metric = tf.keras.metrics.Mean("d_fake_loss", dtype=tf.float32)
d_smooth_loss_metric = tf.keras.metrics.Mean("d_smooth_loss", dtype=tf.float32)
training_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('training_accuracy', dtype=tf.float32)
metric_and_names = [
    (g_total_loss_metric, "g_total_loss"),
    (g_adv_loss_metric, "g_adversarial_loss"),
    (d_total_loss_metric, "d_total_loss"),
    (d_real_loss_metric, "d_real_loss"),
    (d_fake_loss_metric, "d_fake_loss"),
    (d_smooth_loss_metric, "d_smooth_loss"),
    (content_loss_metric, "content_loss")
]

### Training GAN

In [ ]:
import time
from tqdm.notebook import tqdm
%load_ext tensorboard
import datetime, os

def update_metrics(summary_writer,c_loss_tot,g_loss_tot,d_loss_tot,epoch,prec_epochs):
    with summary_writer.as_default():
        for metric, name in metric_and_names:
            if name =="content_loss":
                tf.summary.scalar("generator/"+name, np.mean(c_loss_tot), step=epoch+prec_epochs+1)
            elif name == "g_total_loss":
                tf.summary.scalar("generator/"+name, np.mean(g_loss_tot), step=epoch+prec_epochs+1)
            elif name == "d_total_loss":
                tf.summary.scalar("discriminator/"+name, np.mean(d_loss_tot), step=epoch+prec_epochs+1)
            elif name.startswith("d"):
                tf.summary.scalar("discriminator/"+name, metric.result(), step=epoch+prec_epochs+1)
            else:
                tf.summary.scalar("generator/"+name, metric.result(), step=epoch+prec_epochs+1)
            metric.reset_states()

@tf.function
def train_step(cartoon_set_batch, cartoon_smoothed_batch, normal_photos_batch, generator, discriminator):
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(normal_photos_batch, training=True)

        real_output = discriminator(cartoon_set_batch, training=True)
        fake_output = discriminator(generated_images, training=True)
        real_smooth_output = discriminator(cartoon_smoothed_batch, training=True)
        
        disc_loss = discriminator_loss(real_output, fake_output, real_smooth_output)
        
        gen_adv_loss = generator_loss(fake_output)
        c_loss = mae(vgg(normal_photos_batch), vgg(generated_images))
        gen_total_loss = gen_adv_loss + c_loss*OMEGA #10 =omega
        
        

    gradients_of_generator = gen_tape.gradient(gen_total_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    g_total_loss_metric(gen_total_loss)
    g_adv_loss_metric(gen_adv_loss)
    content_loss_metric(c_loss)
    d_total_loss_metric(disc_loss)

    return disc_loss, gen_total_loss, c_loss 

def train(dataset_name,cartoon_set, cartoon_smoothed_set, normal_photos_set, epochs,prec_epochs, gen_pretrained, D):
    summary_writer = tf.summary.create_file_writer("/content/drive/My Drive/NN/training/"+dataset_name+"/"+TIME+"_logs/")
    with summary_writer.as_default():
        img = np.expand_dims(show_and_save_images(tf.cast((normal_photos_set.next() + 1) * 127.5, tf.uint8),image_name="normal_sample_images.png"), 0,)
        tf.summary.image("normal_images_"+dataset_name, img, step=0)
        img = np.expand_dims(show_and_save_images(tf.cast((cartoon_set.next() + 1) * 127.5, tf.uint8),
            image_name="cartoon_sample_images_"+dataset_name+".png"), 0,)
        tf.summary.image("cartoon_sample_images_"+dataset_name, img, step=0)


    for epoch in tqdm(range(0,epochs), total = epochs):
        start = time.time()
        d_loss_tot,g_loss_tot,c_loss_tot = [],[],[]
        for step in tqdm(range(1, steps_per_epoch + 1), desc=f'Train {epoch + 1+prec_epochs}/{epochs+prec_epochs}', total=steps_per_epoch):
            cart_batch = cartoon_set.next()
            cart_smooth_batch = cartoon_smoothed_set.next()
            norm_photos_batch = normal_photos_set.next()

            d_loss,g_loss,c_loss = train_step(cart_batch, cart_smooth_batch, norm_photos_batch,gen_pretrained,D)
            d_loss_tot.append(d_loss)
            g_loss_tot.append(g_loss)
            c_loss_tot.append(c_loss)
        
        update_metrics(summary_writer,c_loss_tot,g_loss_tot,d_loss_tot,epoch,prec_epochs)
        #print("Generated images at epoch "f"{epoch+1+prec_epochs}")
        fake_batch = tf.cast( (gen_pretrained(SEED, training=False) + 1) * 127.5, tf.uint8)
        img = np.expand_dims(show_and_save_images(
                fake_batch,
                image_name=("gan_generated_images_"+dataset_name+"_at_epoch_"f"{epoch+1+prec_epochs}_"+TIME+".png")
                ),0,
            )
        with summary_writer.as_default():
            tf.summary.image('gan_generated_images_'+dataset_name, img, step=epoch+1+prec_epochs)
        #display  normal
        
        #print("Normal images at epoch "f"{epoch+1+prec_epochs}")
        if epoch==0: 
            show_and_save_images(tf.cast((SEED+1) * 127.5, tf.uint8), image_name=("normal_images_"+dataset_name+"_"+TIME+".png"),to_be_saved=True)
        else:
            show_and_save_images(tf.cast((SEED+1) * 127.5, tf.uint8), image_name=("normal_images_"+dataset_name+"_"+TIME+".png"),to_be_saved=False)
            if (epoch+prec_epochs)% CHECK_AFTER_X_EPOCH == 0:
                savemodel(D,MODEL_NAME_DISC+"_checkpoint_"+str(epoch+prec_epochs)+"ep",checkpoint=True)
                savemodel(gen_pretrained,MODEL_NAME_GEN+"_checkpoint_"+str(epoch+prec_epochs)+"ep",checkpoint=True)

        print("Epoch:  [%5d/%5d] time: %4.4f content_loss: %.8f" % (epoch+1+prec_epochs, epochs+prec_epochs, time.time() - start, np.mean(c_loss_tot)))
        print("Epoch:  [%5d/%5d] time: %4.4f disc_loss: %.8f, gen_loss: %.8f" % (epoch+1+prec_epochs, epochs+prec_epochs, time.time() - start, np.mean(d_loss_tot), np.mean(g_loss_tot)))

    # Generate after the final epoch
    return gen_pretrained, D

def loads_models(model_name_gen="",model_name_disc="",checkpoint=False):
    if EPOCHS_YET_TRAINED>0:
        print("loading")
        gen_pretrained = loadmodel(model_name_gen,checkpoint)
        D = loadmodel(model_name_disc,checkpoint)
    else:
        gen_pretrained = loadmodel("initialization_phase_generator")
        D = create_discriminator(input_shape=(256,256,3))
    return gen_pretrained,D


DATASET_NAME = "paprika"
EPOCHS = 20
EPOCHS_YET_TRAINED = 120
OMEGA = 0.000005
CHECKPOINT = True
CHECK_AFTER_X_EPOCH = 10
TIME = time.strftime("%d-%m_%H-%M", time.gmtime())
MODEL_NAME_DISC = DATASET_NAME+'_discriminator_'+str(EPOCHS_YET_TRAINED+EPOCHS)+"ep_"+TIME
MODEL_NAME_GEN = DATASET_NAME+'_generator_'+str(EPOCHS_YET_TRAINED+EPOCHS)+"ep_"+TIME

if CHECKPOINT:
    MODEL_NAME_DISC = "paprika_discriminator_140ep_10-02_09-42_checkpoint_120ep"
    MODEL_NAME_GEN = "paprika_generator_140ep_10-02_09-42_checkpoint_120ep"
    gen_pretrained, D = loads_models(MODEL_NAME_GEN,MODEL_NAME_DISC,CHECKPOINT)
    if "check" in MODEL_NAME_DISC:
        TIME = MODEL_NAME_DISC[MODEL_NAME_DISC.find("p_")+2:MODEL_NAME_DISC.find("_c")]
    else:
        TIME = MODEL_NAME_DISC[MODEL_NAME_DISC.find("p_")+2:]
else:
    gen_pretrained, D = loads_models()

cartoon_set, cartoon_smoothed_set, normal_photos_set, steps_per_epoch , dataset_kaggle = get_batches(DATASET_NAME)
SEED = dataset_kaggle.next()

%tensorboard --logdir_spec={DATASET_NAME}:"/content/drive/My Drive/NN/training/{DATASET_NAME}/{TIME}_logs" 

generator_trained, discriminator_trained = train(DATASET_NAME,cartoon_set, cartoon_smoothed_set, normal_photos_set, EPOCHS,EPOCHS_YET_TRAINED, gen_pretrained, D)


savemodel(discriminator_trained,MODEL_NAME_DISC)
savemodel(generator_trained,MODEL_NAME_GEN)

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
loading



Model loaded successfully from file /content/drive/My Drive/NN/models/checkpoint/paprika_generator_140ep_10-02_09-42_checkpoint_120ep.h5




Model loaded successfully from file /content/drive/My Drive/NN/models/checkpoint/paprika_discriminator_140ep_10-02_09-42_checkpoint_120ep.h5

Found 3591  images in datasets/paprika/paprika_resized folder.
Found 3592  images in datasets/paprika/paprika_resized_smoothed folder.
Found 2217  images in datasets/normal_photos/photos_from_COCO_resized folder.
Found 3747  images in datasets/normal_photos/photos_from_kaggle_resized folder.


<IPython.core.display.Javascript object>

#### Create gif

In [ ]:
#generator_trained.save_weights(os.path.join(models_dir, "generator_your_name_15"))
#%tensorboard --logdir "/content/drive/My Drive/NN/training/logs_your_name"
#!pip install git+https://github.com/tensorflow/docs
import imageio
import tensorflow_docs.vis.embed as embed
from glob import glob
def create_gif(dataset_name):
    anim_file = 'dcgan'+dataset_name+'.gif'
    with imageio.get_writer(anim_file, mode='I') as writer:
        filenames = glob('/content/drive/My Drive/NN/training/'+dataset_name+'_samples/gan_generated_images_'+dataset_name+'_at_epoch_*_'+TIME+'.png')
        filenames = sorted(filenames)
        for filename in filenames:
            image = imageio.imread(filename)
            writer.append_data(image)
        #image = imageio.imread(filename)
        #writer.append_data(image)
    return anim_file

anim_file = create_gif(dataset_name)
embed.embed_file(anim_file)     

#### test

In [ ]:
#test_gen = loadmodel(MODEL_NAME_GEN)
#test_disc = loadmodel(MODEL_NAME_DISC)
#show_and_save_images(tf.cast((test_gen(SEED)+1) * 127.5, tf.uint8), image_name=("testing.png"), )
#show_and_save_images(tf.cast((test_disc.(SEED)+1) * 127.5, tf.uint8), image_name=("testing.png"), )
%tensorboard --logdir_spec=spirited_away:"/content/drive/My Drive/NN/training/logs_spirited_away_09-02_09-58",your_name:"/content/drive/My Drive/NN/training/logs_your_name" #DATASET_NAME +_TIME


paprika_discriminator_0+1ep_10-02_09-25 paprika_generator_0+1ep_10-02_09-25
